In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Initial Setup
Reading in the datafile and doing some updates to allow for better analysis

In [2]:
# read in initial data
df = pd.read_csv("data/sample_data.csv")
df.head(2)

,Year,Quarter,Month,Product,Revenue,Device,Format,Company
0,2019,Qtr 3,September,Core,29698,Mobile,Display,Apple
1,2019,Qtr 3,September,Core,4171,Desktop,Display,Apple


In [3]:
# setup columns as all lowercase
df.columns = df.columns.str.lower()

In [4]:
# create month number column
df['month_num'] = [datetime.strptime(x, '%B').month for x in df['month']]
df.head(2)

,year,quarter,month,product,revenue,device,format,company,month_num
0,2019,Qtr 3,September,Core,29698,Mobile,Display,Apple,9
1,2019,Qtr 3,September,Core,4171,Desktop,Display,Apple,9


In [5]:
# create datetime formatted column, combining year and month
df['date'] = pd.to_datetime(df['year'].astype(str) + '/' + df['month_num'].astype(str) + '/01', format="%Y/%m")

print(f"The date column is now datatype: {df['date'].dtypes} \n")
print(f"Current column order is: \n{df.columns}")

The date column is now datatype: datetime64[ns] 

Current column order is: 
Index(['year', 'quarter', 'month', 'product', 'revenue', 'device', 'format',
       'company', 'month_num', 'date'],
      dtype='object')


In [6]:
# reorder columns to group date fields (only readability reasons)
df = df[['date', 'year', 'quarter', 'month', 'month_num', 'company', 'product', 'device', 'format', 'revenue']]
df.head()

,date,year,quarter,month,month_num,company,product,device,format,revenue
0,2019-09-01,2019,Qtr 3,September,9,Apple,Core,Mobile,Display,29698
1,2019-09-01,2019,Qtr 3,September,9,Apple,Core,Desktop,Display,4171
2,2019-08-01,2019,Qtr 3,August,8,Apple,Core,Mobile,Display,88938
3,2019-08-01,2019,Qtr 3,August,8,Apple,Core,Desktop,Display,10154
4,2019-07-01,2019,Qtr 3,July,7,Apple,Core,Mobile,Display,85092


# Questions

## Question 1
In how many quarters did “Orange” have more than $15K in “Social” revenue?

In [7]:
# aggregate sum of revenue by year and quarter
year_qtr_df = df.groupby(['year', 'quarter', 'company', 'product']).agg({'revenue': 'sum'}).reset_index()

# filter by orange and social
orange_soc_df = year_qtr_df.loc[(year_qtr_df['revenue'] > 15000) & 
                (year_qtr_df['product'] == 'Social') &
                (year_qtr_df['company'] == 'Orange')]
orange_soc_df

,year,quarter,company,product,revenue
57,2017,Qtr 4,Orange,Social,27423
77,2018,Qtr 1,Orange,Social,59855
97,2018,Qtr 2,Orange,Social,79679
116,2018,Qtr 3,Orange,Social,75458
135,2018,Qtr 4,Orange,Social,75075
153,2019,Qtr 1,Orange,Social,52670
172,2019,Qtr 2,Orange,Social,75734
190,2019,Qtr 3,Orange,Social,77461


In [8]:
# answer question
orange_soc_count = len(orange_soc_df['revenue'])
print(f'Orange company has had {orange_soc_count} quarters with more than 15K in Social revenue')

Orange company has had 8 quarters with more than 15K in Social revenue


## Question 2
What was the average quarter-over-quarter (QoQ) growth in revenue among
companies from Q2 to Q3 in 2018?

In [9]:
# filter by 2018 quarters 2 and 3
Q2_Q3_2018_df = df[(df['date'].dt.year == 2018) & (df['date'].dt.quarter.isin([2,3]))]
print(f"unique years:  {Q2_Q3_2018_df['year'].unique()}")
print(f"unique quarters:  {Q2_Q3_2018_df['quarter'].unique()}")

unique years:  [2018]
unique quarters:  ['Qtr 3' 'Qtr 2']


In [10]:
# group by company, aggregate revenue
pivot_comp_rev_df = Q2_Q3_2018_df.pivot_table(index='company', columns='quarter', aggfunc={'revenue': 'sum'}).fillna(0)
pivot_comp_rev_df.head(2)

revenue          
quarter     Qtr 2     Qtr 3
company                    
Amazon    73947.0   50096.0
Apple    178898.0  156500.0

In [11]:
# I prefer to flatten the columns as opposed to the multi-index, so have this function I may use after pivoting data
def flatten_cols(df):
    df.columns = [
        '_'.join(tuple(map(str, t))).strip() 
        for t in df.columns.values
        ]
    return df

In [12]:
# flatten index
comp_rev_df = flatten_cols(pivot_comp_rev_df)
comp_rev_df.head(2)

,revenue_Qtr 2,revenue_Qtr 3
company,,
Amazon,73947.0,50096.0
Apple,178898.0,156500.0


In [13]:
# rename columns for readability and to be more explicit
comp_rev_df.columns = ['2018_Q2_revenue', '2018_Q3_revenue']

In [14]:
# add quarter over quarter change by company
comp_rev_df['QoQ_change_perc'] = ((comp_rev_df['2018_Q3_revenue'] / comp_rev_df['2018_Q2_revenue']) -1) * 100
comp_rev_df

,2018_Q2_revenue,2018_Q3_revenue,QoQ_change_perc
company,,,
Amazon,73947.0,50096.0,-32.254182
Apple,178898.0,156500.0,-12.519983
Ganges,147428.0,137777.0,-6.546246
Grape,194384.0,0.0,-100.000000
Kiwi,110770.0,134222.0,21.171797
Mississippi,128858.0,77362.0,-39.963371
Nile,49581.0,249745.0,403.711099
Orange,138005.0,165222.0,19.721749
Pear,40028.0,568973.0,1321.437494


In [15]:
# stats on quarter over quarter performance
QoQ_stats = comp_rev_df['QoQ_change_perc'].describe()
QoQ_stats

count      10.000000
mean      158.538277
std       430.970858
min      -100.000000
25%       -27.320632
50%         2.039084
75%        20.809285
max      1321.437494
Name: QoQ_change_perc, dtype: float64

In [16]:
# answer question

# average growth
print(f"The average growth of revenue by company from Q2-2018 to Q3-2018 was {QoQ_stats['mean'].round(1)}%")

# overall growth
overall_growth = (sum(comp_rev_df['2018_Q3_revenue']) / sum(comp_rev_df['2018_Q2_revenue']) -1 )* 100
print(f"The overall revenue growth from Q2-2018 to Q3-2018 was {round(overall_growth, 1)}%")

The average growth of revenue by company from Q2-2018 to Q3-2018 was 158.5%
The overall revenue growth from Q2-2018 to Q3-2018 was 36.7%


## Question 3
List the companies by the number of months they have spent as a client, in
descending order.


In [17]:
# answer question

# group by company and aggregate date by count
month_count_df = df.groupby(['company']).agg({'date': 'count'})
month_count_df = month_count_df.rename(columns={'date': 'count_of_months'})
month_count_df.sort_values(by='count_of_months', ascending=False).reset_index()

,company,count_of_months
0,Kiwi,255
1,Ganges,218
2,Amazon,216
3,Orange,205
4,Mississippi,201
5,Yangtze,66
6,Apple,52
7,Pear,16
8,Nile,15
9,Grape,7


## Question 4
In each year, what percentage of “Core” revenue has come from video?

In [18]:
# filter by only core product
core_df = df.loc[(df['product'] == "Core")]
core_df.head(2)

,date,year,quarter,month,month_num,company,product,device,format,revenue
0,2019-09-01,2019,Qtr 3,September,9,Apple,Core,Mobile,Display,29698
1,2019-09-01,2019,Qtr 3,September,9,Apple,Core,Desktop,Display,4171


In [19]:
# group by company, aggregate revenue
pivot_year_core_df = core_df.pivot_table(index='year', columns='format', aggfunc={'revenue': 'sum'}).fillna(0)

# flatten index
year_core_df = flatten_cols(pivot_year_core_df)

# add a total revenue column
year_core_df['core_revenue_total'] = year_core_df.sum(numeric_only=True, axis=1)
year_core_df

,revenue_Display,revenue_Video,core_revenue_total
year,,,
2017,3213743,14975,3228718
2018,6453001,22784,6475785
2019,3912259,25137,3937396


In [20]:
# convert to percentages
year_core_perc_df = year_core_df.div(year_core_df['core_revenue_total'], axis=0) * 100 
year_core_perc_df = year_core_perc_df.reset_index()
year_core_perc_df

,year,revenue_Display,revenue_Video,core_revenue_total
0,2017,99.536194,0.463806,100.0
1,2018,99.648166,0.351834,100.0
2,2019,99.361583,0.638417,100.0


In [21]:
# answer question
for row in year_core_perc_df.itertuples():
    print(f"In {row.year} Core product had a {round(row.revenue_Video, 2)}% of its revenue come from video")

print("\nOverall, a fairly small percentage at below 1% in each year")

In 2017 Core product had a 0.46% of its revenue come from video
In 2018 Core product had a 0.35% of its revenue come from video
In 2019 Core product had a 0.64% of its revenue come from video

Overall, a fairly small percentage at below 1% in each year


## Question 5
Which month saw the highest month-over-month (MoM) percent change in total
revenue?

In [22]:
# aggregate revenue by month
month_rev_df = df.groupby('date').agg({'revenue': 'sum'})
month_rev_df.head()

,revenue
date,
2017-01-01,188665
2017-02-01,230227
2017-03-01,304013
2017-04-01,195051
2017-05-01,243337


In [33]:
# create percent change column
month_rev_df['percent_change'] = (month_rev_df['revenue'].pct_change()) * 100
month_rev_df.head()

,revenue,percent_change
date,,
2017-01-01,188665,NaN
2017-02-01,230227,22.029523
2017-03-01,304013,32.049238
2017-04-01,195051,-35.841230
2017-05-01,243337,24.755577


In [34]:
# stats on percent change by month
max_revenue_chg = month_rev_df['percent_change'].max()

# year and month with highest revenue change
highest_month_chg = month_rev_df['percent_change'].idxmax().strftime("%Y-%m")

In [35]:
# answer question

print(f"{highest_month_chg} had the largest MoM revenue growth at {round(max_revenue_chg, 1)}%")

2017-12 had the largest MoM revenue growth at 60.7%


# Additional Analysis

In [26]:
# use pandas profiling package (to view you may need to pip install pandas-profiling)
# import pandas_profiling

# prof_report = pandas_profiling.ProfileReport(df , title = 'Summary Profile')
# prof_report.to_widgets()